<a href="https://colab.research.google.com/github/jurandifr/Python/blob/main/MapaAstral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install kerykeion cairosvg fpdf2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.9/52.9 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.1/379.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.4/499.4 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.7 MB/s eta 0:00:00
  Created wheel for scour: filename=scour-0.38.2-py3-none-any.whl size=54248 sha256=4a1f3a36f70e2ba5133521a57ae2b6eb2a6db9e0fcf250fd5ed15c902dcc1d8c
  Stored in directo

In [2]:
# Reiniciar o ambiente parece mais eficiente que retentar... talver reiniciar o modelo possa ser uma boa opção

# Suprimir warnings
from absl import logging
logging.set_verbosity(logging.ERROR)

# Faz o Mapa
from kerykeion import AstrologicalSubject, KerykeionChartSVG

# Faz a interpretação
import google.generativeai as genai # 15/min
from google.colab import userdata
genai.configure(api_key=userdata.get('geminikey'))
model = genai.GenerativeModel("gemini-1.5-flash")
from time import sleep
import json
from tqdm.notebook import tqdm

# Corrige o SVG
from bs4 import BeautifulSoup

# Gera o PDF
import cairosvg
from fpdf import FPDF, XPos, YPos
import re



In [3]:
def corrigeSVG(input_svg_path):
  # Seu dicionário de cores
  cores = {
  '--kerykeion-chart-color-paper-0' : '#000000',
  '--kerykeion-chart-color-paper-1' : '#ffffff',
  '--kerykeion-chart-color-zodiac-bg-0' : '#ff7200',
  '--kerykeion-chart-color-zodiac-bg-1' : '#6b3d00',
  '--kerykeion-chart-color-zodiac-bg-2' : '#69acf1',
  '--kerykeion-chart-color-zodiac-bg-3' : '#2b4972',
  '--kerykeion-chart-color-zodiac-bg-4' : '#ff7200',
  '--kerykeion-chart-color-zodiac-bg-5' : '#6b3d00',
  '--kerykeion-chart-color-zodiac-bg-6' : '#69acf1',
  '--kerykeion-chart-color-zodiac-bg-7' : '#2b4972',
  '--kerykeion-chart-color-zodiac-bg-8' : '#ff7200',
  '--kerykeion-chart-color-zodiac-bg-9' : '#6b3d00',
  '--kerykeion-chart-color-zodiac-bg-10' : '#69acf1',
  '--kerykeion-chart-color-zodiac-bg-11' : '#2b4972',
  '--kerykeion-chart-color-zodiac-icon-0' : '#ff7200',
  '--kerykeion-chart-color-zodiac-icon-1' : '#6b3d00',
  '--kerykeion-chart-color-zodiac-icon-2' : '#69acf1',
  '--kerykeion-chart-color-zodiac-icon-3' : '#2b4972',
  '--kerykeion-chart-color-zodiac-icon-4' : '#ff7200',
  '--kerykeion-chart-color-zodiac-icon-5' : '#6b3d00',
  '--kerykeion-chart-color-zodiac-icon-6' : '#69acf1',
  '--kerykeion-chart-color-zodiac-icon-7' : '#2b4972',
  '--kerykeion-chart-color-zodiac-icon-8' : '#ff7200',
  '--kerykeion-chart-color-zodiac-icon-9' : '#6b3d00',
  '--kerykeion-chart-color-zodiac-icon-10' : '#69acf1',
  '--kerykeion-chart-color-zodiac-icon-11' : '#2b4972',
  '--kerykeion-chart-color-zodiac-radix-ring-0' : '#ff0000',
  '--kerykeion-chart-color-zodiac-radix-ring-1' : '#ff0000',
  '--kerykeion-chart-color-zodiac-radix-ring-2' : '#ff0000',
  '--kerykeion-chart-color-zodiac-transit-ring-0' : '#ff0000',
  '--kerykeion-chart-color-zodiac-transit-ring-1' : '#ff0000',
  '--kerykeion-chart-color-zodiac-transit-ring-2' : '#0000ff',
  '--kerykeion-chart-color-zodiac-transit-ring-3' : '#0000ff',
  '--kerykeion-chart-color-houses-radix-line' : '#ff0000',
  '--kerykeion-chart-color-houses-transit-line' : '#0000ff',
  '--kerykeion-chart-color-lunar-phase-0' : '#000000',
  '--kerykeion-chart-color-lunar-phase-1' : '#ffffff',
  # Aspects
  '--kerykeion-chart-color-conjunction' : '#5757e2',
  '--kerykeion-chart-color-semi-sextile' : '#810757',
  '--kerykeion-chart-color-semi-square' : '#b14e58',
  '--kerykeion-chart-color-sextile' : '#d59e28',
  '--kerykeion-chart-color-quintile' : '#1f99b3',
  '--kerykeion-chart-color-square' : '#dc0000',
  '--kerykeion-chart-color-trine' : '#36d100',
  '--kerykeion-chart-color-sesquiquadrate' : '#985a10',
  '--kerykeion-chart-color-biquintile' : '#7a9810',
  '--kerykeion-chart-color-quincunx' : '#26bbcf',
  '--kerykeion-chart-color-opposition' : '#510060',
  # Planets
  '--kerykeion-chart-color-sun' : '#984b00',
  '--kerykeion-chart-color-moon' : '#150052',
  '--kerykeion-chart-color-mercury' : '#520800',
  '--kerykeion-chart-color-venus' : '#400052',
  '--kerykeion-chart-color-mars' : '#540000',
  '--kerykeion-chart-color-jupiter' : '#47133d',
  '--kerykeion-chart-color-saturn' : '#124500',
  '--kerykeion-chart-color-uranus' : '#6f0766',
  '--kerykeion-chart-color-neptune' : '#06537f',
  '--kerykeion-chart-color-pluto' : '#713f04',
  '--kerykeion-chart-color-mean-node' : '#4c1541',
  '--kerykeion-chart-color-true-node' : '#4c1541',
  '--kerykeion-chart-color-chiron' : '#666f06',
  '--kerykeion-chart-color-first-house' : '#ff7e00',
  '--kerykeion-chart-color-tenth-house' : '#ff0000',
  '--kerykeion-chart-color-seventh-house' : '#0000ff',
  '--kerykeion-chart-color-fourth-house' : '#000000',
  '--kerykeion-chart-color-mean-lilith' : '#000000',
  # Elements Percentage
  '--kerykeion-chart-color-air-percentage' : '#6f76d1',
  '--kerykeion-chart-color-earth-percentage' : '#6a2d04',
  '--kerykeion-chart-color-fire-percentage' : '#ff6600',
  '--kerykeion-chart-color-water-percentage' : '#630e73',
  # Other
  '--kerykeion-chart-color-house-number' : '#f00'
  }

  # Carregar o arquivo SVG
  with open(input_svg_path, 'r') as file:
      svg_content = file.read()

  # Parsear o conteúdo SVG
  soup = BeautifulSoup(svg_content, 'xml')

  # Substituir variáveis CSS por cores fixas
  for style in soup.find_all('style'):
      css_content = style.string
      for variavel, cor in cores.items():
          css_content = css_content.replace(f'var({variavel})', cor)
      style.string.replace_with(css_content)

  # Substituir variáveis CSS nos atributos dos elementos
  for element in soup.find_all(True):  # True encontra todos os elementos
      for attr, value in element.attrs.items():
          if isinstance(value, str):
              for variavel, cor in cores.items():
                  if f'var({variavel})' in value:
                      element[attr] = value.replace(f'var({variavel})', cor)


  # Salvar o SVG modificado
  with open(input_svg_path, 'w') as file:
      file.write(str(soup))

def filter_dict(original_dict):
    keys = ['name', 'sign', 'house', 'retrograde']
    planets = ['Sun', 'Moon', 'Mercury','Venus','Mars','Jupiter','Saturn',
      'Uranus','Neptune','Pluto','Mean_Node','True_Node','Mean_South_Node',
      'True_South_Node','Chiron','Mean_Lilith']
    temp = {planet: {key : original_dict[planet.lower()][key] for key in keys} for planet in planets}
    temp['Rising_Sign'] = { 'name': 'Rising_Sign',
                      'sign' :original_dict['first_house']['sign'] }
    return temp

class PDF(FPDF):
    def header(self):
        self.set_font('Helvetica', 'B', 12)
        self.set_text_color(0, 0, 0)
        self.cell(0, 10, 'Mapa Astral', 0, new_x=XPos.LMARGIN, new_y=YPos.NEXT, align='C')

    def chapter_title(self, title):
        self.set_font('Helvetica', 'B', 14)
        self.set_text_color(0, 200, 0)
        self.cell(0, 10, title, 0, new_x=XPos.LMARGIN, new_y=YPos.NEXT)
        self.set_y(self.get_y() + 10)

    def add_text_with_bold(self, text):
        bold_pattern = re.compile(r'\*\*(.*?)\*\*')
        parts = bold_pattern.split(text)

        for i, part in enumerate(parts):
            if i % 2 == 0:
                self.set_font("Helvetica", size=12)
            else:
                self.set_font("Helvetica", style='B', size=12)
            self.set_text_color(0, 0, 0)
            self.multi_cell(0, 4, part)
            self.set_y(self.get_y() + 1)

    def add_png(self, png_path):
        self.image(png_path, x=10, y=self.get_y(), w=190)
        self.set_y(self.get_y() + 130)

def geraPDF(fileSvg, data):
    # Converter SVG para PNG
    cairosvg.svg2png(url=fileSvg, write_to='mapa.png')

    pdf = PDF()
    pdf.add_page()

    # Adicionando a imagem PNG
    pdf.add_png('mapa.png')

    # Substituir caracteres Unicode por ASCII
    def replace_unicode(text):
      unicode_map = str.maketrans({
        '–': '-',
        '“': "'",
        '”': "'",
        '\u2013' : '-'
      })
      return text.translate(unicode_map)

    # Adicionando títulos e textos
    for i in data:
        title = replace_unicode(i[0])
        body = replace_unicode(i[1])
        pdf.chapter_title(title)
        pdf.add_text_with_bold(body)

    # Salvando o PDF
    pdf.output('documento.pdf')


def mapa(nome, dia,mes,ano,hora,minuto,lat,lon, city= "São Paulo"):

  fileSvg = f"{nome} - Natal Chart.svg"

  subject = AstrologicalSubject( nome , ano, mes, dia, hora, minuto,
                                lat=lat, lng=lon,
                                tz_str='America/Sao_Paulo', city=city,
                                online=False)
  KerykeionChartSVG(subject, new_output_directory=".", chart_language='PT').makeSVG(minify=True)
  corrigeSVG(fileSvg)

  data = []
  for key,info in tqdm(filter_dict(json.loads(subject.json())).items()):
    ask = f"Na astrologia, explique o que significa: \n {info}"
    ask = ask.replace("{","").replace("}","")
    response = model.generate_content(ask)
    data.append((info['name'], response.text))
    sleep(6)

  geraPDF(fileSvg,data)

In [4]:
# dia = 15
# mes = 5
# ano = 1985
# hora = 3
# minuto = 45
# gmt = -3  # tz_str='America/Sao_Paulo
# lat = -23.5505  # Latitude de São Paulo
# lon = -46.6333  # Longitude de São Paulo
# #lat = -23.46278 #GRU
# #lon = 46.53306 #GRU
# nome = "Flavia Rodrigues"

# mapa(nome, dia,mes,ano,hora,minuto,lat,lon)
#Latitude: -25.4232 Longitude: -49.2710. Curitiba ( Tupi: "Pine Nut Land", Portuguese pronunciation: [kuɾiˈtʃibɐ]) is the capital and largest city of the Brazilian state of Paraná.
# Taboao da Serra: -23.6228759. Longitude de Taboao da Serra: -46.7816647.

In [7]:
dia = 1
mes = 1
ano = 2000
hora = 1
minuto = 1
gmt = -3  # tz_str='America/Sao_Paulo
lat = -23.5505
lon = -46.6333
city = "São Paulo"
nome = "Pessoa"

mapa(nome, dia,mes,ano,hora,minuto,lat,lon, city=city)

  0%|          | 0/17 [00:00<?, ?it/s]